# Import and Inizialization

In [2]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
from heapq import heappop, heappush
from typing import Tuple, Union

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

In [ ]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

Function to evaluate the quality of a solution (list of actions) as the total number of actions needed.

In [ ]:
def qualily(actions):
    return len(actions)

The state is a numpy array.

We created a function that returns the number of actions from a state pos1 to a state pos2.

Compute 100_000 random actions:

In [ ]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing: 100%|██████████| 100000/100000 [00:01<00:00, 62482.08it/s]


array([[ 9, 44, 18, 12, 40, 39, 38],
       [ 1, 15,  8,  2,  4,  0, 10],
       [30, 46, 48, 16, 23, 27, 17],
       [26, 47, 33, 20, 45, 32, 34],
       [42, 22, 25, 35, 29, 24, 41],
       [43, 21, 28, 31, 14, 19, 13],
       [11, 37,  3,  6,  5,  7, 36]])

Define a function that indicates if we end the search.

In [ ]:
def test_goal(solution):
    arr_solution = np.reshape(solution, PUZZLE_DIM*PUZZLE_DIM)
    arr_solution_no_zero = arr_solution[0: len(arr_solution)-1]
    if np.all(arr_solution_no_zero[:-1] <= arr_solution_no_zero[1:]) and arr_solution[len(arr_solution)-1]==0:
        return True
    return False

In [ ]:
def depth_limited_search(initial_state: np.ndarray, final_state: np.ndarray, max_depth: int) -> list[action] or None:
    stack = deque([(initial_state, [], 0)])  # Stack of (current state, path to reach it, current depth)
    visited = set()  # Set of visited states for current path only
    optimum = matrix_score(final_state)

    while stack:
        current_state, path, depth = stack.pop()
        current_score = matrix_score(current_state)

        # Check if we reached the goal
        if current_score == optimum:
            return path
        
        # # Backtrack if depth limit reached
        if depth >= max_depth:
            continue
        
        # Add current state to visited set (track only in current path)
        visited.add(current_score)
        
        # Generate and iterate over all possible moves
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            
            # Check if the next state has already been visited in the current path
            if matrix_score(next_state) not in visited:
                # Add the new state and path to stack, increase depth
                stack.append((next_state, path + [act], depth + 1))
        
        # Remove the current state from visited set after backtracking
        #visited.remove(matrix_score(current_state))
    
    return None  # Return None if no solution is found within depth limit

# Iterative Deepening Depth-First Search (IDDFS) wrapper function
def iterative_deepening_dfs(initial_state: np.ndarray, final_state: np.ndarray, max_depth: int = 50) -> list[action] or None:
    for depth in range(1, max_depth + 1):
        result = depth_limited_search(initial_state, final_state, depth)
        if result is not None:
            return result
    return None


In [ ]:
test_goal = np.arange(1, PUZZLE_DIM*PUZZLE_DIM, 1)
test_goal = np.append(test_goal, 0)
test_goal = test_goal.reshape((PUZZLE_DIM, PUZZLE_DIM))

In [ ]:
actions, costValue = depth_limited_search(state, test_goal)

In [ ]:
qualily(actions)

620

In [ ]:
costValue

70818.0

In [ ]:
state

array([[ 9, 44, 18, 12, 40, 39, 38],
       [ 1, 15,  8,  2,  4,  0, 10],
       [30, 46, 48, 16, 23, 27, 17],
       [26, 47, 33, 20, 45, 32, 34],
       [42, 22, 25, 35, 29, 24, 41],
       [43, 21, 28, 31, 14, 19, 13],
       [11, 37,  3,  6,  5,  7, 36]])

In [ ]:
test_goal

array([[ 1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19, 20, 21],
       [22, 23, 24, 25, 26, 27, 28],
       [29, 30, 31, 32, 33, 34, 35],
       [36, 37, 38, 39, 40, 41, 42],
       [43, 44, 45, 46, 47, 48,  0]])